In [2]:
import pandas as pd
import requests
import io
import logging
import traceback
from typing import Optional, List, Dict
from openpyxl import load_workbook
from datetime import datetime
from openai import OpenAI
import urllib.parse
from msal import ConfidentialClientApplication
import os
import shutil
import json
import urllib.parse

In [ ]:
import pandas as pd
import requests
import time
from msal import ConfidentialClientApplication
import os
from tqdm import tqdm

# ============================================
#             CONFIGURATION
# ============================================
# Configuration details
CONFIG = {
    'client_secret': os.getenv("CLIENT_SECRET"),
    'client_id': os.getenv("CLIENT_ID"),
    'tenant_id': os.getenv("TENANT_ID"),
    'bcc_email': os.getenv("BCC_EMAIL"),
    'drive_id': os.getenv("DRIVE_ID"),
    'site_id': os.getenv("SITE_ID"),
    'file_id': os.getenv("FILE_ID"),
    'hubspot_api_key': os.getenv("HUBSPOT_API_KEY"),  
}

client_id = CONFIG['client_id']
tenant_id = CONFIG['tenant_id']
client_secret = CONFIG['client_secret']
drive_id = CONFIG['drive_id']
file_id = CONFIG['file_id']
hubspot_api_key = CONFIG['hubspot_api_key']
bcc_email = CONFIG['bcc_email']

DEFAULT_EMAIL = """ 
<html>
<body style="color: black; font-family: Aptos, Arial, sans-serif; font-size: 12pt; line-height: 1.5;">
    <p style="margin-bottom: 16px;">Dear {first_name},</p>

    <p style="margin-bottom: 16px;">FuelEU pooling is now trading <strong>around EUR 200</strong>— a cheaper, more flexible way than blending biofuels yourself. With FuelEU penalties priced at 3x that level, the choice is clear. </p>

    <p style="margin-bottom: 16px;">Over the past two quarters, we’ve coordinated some of the first FuelEU pooling deals, offering flexible volumes to accommodate <u>uncertainty around year-end emissions.</u> </p>

    <p style="margin-bottom: 16px;">Here’s how we've supported our clients: </p>
    <ul style="margin-bottom: 16px;">
        <li><strong>Cut compliance costs </strong>via pooling</li>
        <li>Access<strong> flexible volumes</strong> and a <strong>fixed-price </strong></li>
        <li>Connect with <strong>trusted counterparties </strong></li>
    </ul>

    <p style="margin-bottom: 16px;">With 50+ years of combined experience in fuel and environmental commodity trading, we work <i>independently </i>to secure the best deal for our clients.  </p>

    <p style="margin-bottom: 16px;">Would you be open to a quick 15-minute call next week to explore options? If timing’s not right, I can send over a short overview instead. </p>

    <p style="margin-bottom: 16px;">Kind regards,</p>

<!-- Signature starts here -->
    <table style="font-family: Aptos, Arial, sans-serif; font-size: 10pt; margin-top: 16px;" cellspacing="0" cellpadding="0">
    <tr>
        <!-- Name Cell -->
        <td style="padding-right: 50px; vertical-align: middle; height: 80px;">
            <strong style="font-size: 11pt;">Meesz Niehe</strong>
        </td>

        <!-- Logo Cell -->
        <td>
            <img src="https://cdn.prod.website-files.com/662106f5bfeef405750dfdb1/662106f5bfeef405750dfde6_Complete_dark_short.webp" alt="Carbon Leap" style="height:80px;">
        </td>
    </tr>

    <!-- Separator Line -->
    <tr>
        <td colspan="2">
            <hr style="border: 0; border-top: 1px solid #ccc; margin: 8px 0;">
        </td>
    </tr>

    <!-- Contact Details -->
    <tr>
        <td style="padding-right: 50px; vertical-align: top;">
            <span style="color: #B0B0B0;">Open for a call:</span> <span style="color: #1886D8;">+31 6 18868417</span><br>
            <span style="color: #B0B0B0;">Send an email:</span> 
            <a href="mailto:meeszniehe@carbonleap.nl" style="color: #1886D8; text-decoration: none;">meeszniehe@carbonleap.nl</a><br>
            <span style="color: #B0B0B0;">Visit our website:</span> 
            <a href="https://www.carbonleap.nl" style="color: #1886D8; text-decoration: none;">www.carbonleap.nl</a><br>
        </td>
        <td style="vertical-align: top; color: #A0A0A0;">
            Laan van Kronenburg 14<br>
            1183 AS Amstelveen<br>
            The Netherlands
        </td>
    </tr>
</table>
<p style="font-size: 10pt; color: #A0A0A0; margin-top: 20px; font-family: Aptos, Arial, sans-serif;">
    <i>At Carbonleap, we’ve facilitated many of the first FuelEU pooling agreements — connecting ship operators with compliance deficits to vetted surplus holders from our trusted network of suppliers. With a dedicated team on FuelEU and combined 50+ years of experience in fuel and environmental commodity trading, we manage the full process: from sourcing from reliable partners and negotiating fair terms, to administrative and reporting obligations.</i>
</p>

</body>
</html>

"""



def authenticate_client(client_id: str, tenant_id: str, client_secret: str) -> str:
    """
    Authenticate with Microsoft Graph API using Client Credentials Flow.
    Returns access token.
    """
    try:
        authority = f'https://login.microsoftonline.com/{tenant_id}'
        app = ConfidentialClientApplication(
            client_id=client_id,
            authority=authority,
            client_credential=client_secret
        )
        token_response = app.acquire_token_for_client(scopes=['https://graph.microsoft.com/.default'])

        if 'access_token' in token_response:
            print("Successfully acquired access token.")
            return token_response['access_token']
        else:
            raise Exception("Failed to acquire access token.")
    except Exception as e:
        print(f"Authentication failed: {str(e)}")
        raise

def create_draft_email(access_token: str, sender_email: str, recipient_email: str, subject: str, body: str) -> bool:
    """
    Create an email draft in the sender's Microsoft Outlook inbox.
    """
    endpoint = f"https://graph.microsoft.com/v1.0/users/{sender_email}/messages"
    email_msg = {
        "subject": subject,
        "body": {"contentType": "HTML", "content": body},
        "toRecipients": [{"emailAddress": {"address": recipient_email}}],
        "ccRecipients": [{"emailAddress": {"address": cc_email}}],
        "bccRecipients": [{"emailAddress": {"address": bcc_email}}],  
        "isDraft": True
    }

    headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}

    print(f"Sending email draft to Outlook API: {recipient_email} - {subject}")

    response = requests.post(endpoint, headers=headers, json=email_msg)

    print(f"Response Code: {response.status_code}")
    print(f"Response Body: {response.text}")  # Debugging API response

    return response.status_code == 201

def read_excel_data(file_path: str):
    """
    Read contact data from Excel file
    """
    try:
        df = pd.read_excel(file_path)
        required_columns = ["Company Name", "First Name", "Last Name", "Job Title", "Email"]
        
        # Check if all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing required columns in Excel file: {missing_columns}")
            return None
            
        # Clean up data
        df = df.dropna(subset=["Email", "First Name"])  # Remove rows with missing emails or first names
        
        print(f"Successfully loaded {len(df)} contacts from Excel file")
        return df
    except Exception as e:
        print(f"Error reading Excel file: {str(e)}")
        return None

def create_personalized_email(row, template=DEFAULT_EMAIL):
    """
    Create personalized email content based on contact info
    """
    try:
        first_name = row["First Name"]
        personalized_email = template.format(first_name=first_name)
        
        
        return personalized_email
    except KeyError as e:
        print(f"Error personalizing email: Missing field {e}")
        return None

def main():
    # Use the sender email you want
    sender_email = "meeszniehe@carbonleap.nl"
    cc_email = "guidolevie@carbonleap.nl"
    
    # Get authentication token
    access_token = authenticate_client(client_id, tenant_id, client_secret)
    
    # Get path to Excel file
    file_path = "Batch1_Meesz.xlsx"
    contacts_df = read_excel_data(file_path)
    
    if contacts_df is None or len(contacts_df) == 0:
        print("No valid contacts found. Exiting.")
        return
    
    # Confirm before proceeding
    print(f"Found {len(contacts_df)} contacts. Preview of first 5:")
    print(contacts_df.head())
    
    proceed = input(f"Do you want to create {len(contacts_df)} draft emails? (y/n): ").lower()
    if proceed != 'y':
        print("Operation canceled by user.")
        return
    
    # Process each contact
    success_count = 0
    error_count = 0
    
    for index, row in tqdm(contacts_df.iterrows(), total=len(contacts_df), desc="Creating drafts"):
        try:
            recipient_email = row["Email"]
            company_name = row.get("Company Name", "")
            first_name = row["First Name"]
            last_name = row["Last Name"]
            
            # Create subject line
            subject = f" Pooling cheaper than using biofuel"
            
            # Create personalized email body
            email_body = create_personalized_email(row)
            
            if email_body:
                # Create draft email
                success = create_draft_email(
                    access_token=access_token,
                    sender_email=sender_email,
                    recipient_email=recipient_email,
                    subject=subject,
                    body=email_body,
                    
                )
                
                if success:
                    success_count += 1
                else:
                    error_count += 1
                    print(f"Failed to create draft for {recipient_email}")
                
                # Add a small delay to avoid rate limiting
                time.sleep(1)
            else:
                error_count += 1
                print(f"Failed to generate email content for {recipient_email}")
                
        except Exception as e:
            error_count += 1
            print(f"Error processing contact {index}: {str(e)}")
    
    print(f"\nCompleted email draft creation:")
    print(f"- Successfully created: {success_count}")
    print(f"- Failed: {error_count}")
    print(f"- Total processed: {success_count + error_count}")

if __name__ == "__main__":
    main()

Successfully acquired access token.
Successfully loaded 28 contacts from Excel file
Found 28 contacts. Preview of first 5:
                    Company Name           First Name            Last Name  \
0            Union Maritime Ltd.  Union Maritime Ltd.  Union Maritime Ltd.   
1      AVIN INTERNATIONAL L.T.D.                 Nina           Karagiorgi   
2                      Saga plc.                 Owen              Balmont   
3  Pleiades Shipping Agents S.A.           Aikaterini              Margeti   
4  Navios Maritime Holdings Inc.            Marianthi             Talaveri   

                                    Job Title                           Email  
0                         Union Maritime Ltd.             Union Maritime Ltd.  
1  Quality Assurance & Compliance Coordinator             nkaragiorgi@avin.gr  
2                                 Head of ESG         owen.balmont@saga.co.uk  
3                            HSQE Coordinator  aikaterini.margeti@pleiades.gr  
4       

Creating drafts:   0%|          | 0/28 [00:00<?, ?it/s]

Sending email draft to Outlook API: Union Maritime Ltd. -  Pooling cheaper than using biofuel
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('meeszniehe%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaTQ\"","id":"AAMkADllYTVkMjNiLWIyMTUtNDJlMi04ZmEzLTc5MWE1Njg1YzFhZgBGAAAAAAAtEZn6fxGeTa1IW7d-z5k1BwCpQwA2oX5_S6qK2pjvDllMAAAAAAEPAACpQwA2oX5_S6qK2pjvDllMAAAYreakAAA=","createdDateTime":"2025-04-24T09:29:51Z","lastModifiedDateTime":"2025-04-24T09:29:51Z","changeKey":"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaTQ","categories":[],"receivedDateTime":"2025-04-24T09:29:51Z","sentDateTime":"2025-04-24T09:29:51Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB50922CB0CE16609CE548A7A7D6852@AM6PR07MB5092.eurprd07.prod.outlook.com>","subject":" Pooling cheaper than using biofuel","bodyPreview":"Dear Union Maritime Ltd.,\r\n\r\nFuelEU pooling is now trading around EUR 200\u2014 a cheaper, more flexib

Creating drafts:   4%|▎         | 1/28 [00:01<00:36,  1.35s/it]

Sending email draft to Outlook API: nkaragiorgi@avin.gr -  Pooling cheaper than using biofuel
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('meeszniehe%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaTS\"","id":"AAMkADllYTVkMjNiLWIyMTUtNDJlMi04ZmEzLTc5MWE1Njg1YzFhZgBGAAAAAAAtEZn6fxGeTa1IW7d-z5k1BwCpQwA2oX5_S6qK2pjvDllMAAAAAAEPAACpQwA2oX5_S6qK2pjvDllMAAAYrealAAA=","createdDateTime":"2025-04-24T09:29:52Z","lastModifiedDateTime":"2025-04-24T09:29:52Z","changeKey":"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaTS","categories":[],"receivedDateTime":"2025-04-24T09:29:52Z","sentDateTime":"2025-04-24T09:29:52Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB5092381F1A6140E1FD25EA01D6852@AM6PR07MB5092.eurprd07.prod.outlook.com>","subject":" Pooling cheaper than using biofuel","bodyPreview":"Dear Nina,\r\n\r\nFuelEU pooling is now trading around EUR 200\u2014 a cheaper, more flexible way than ble

...

Sending email draft to Outlook API: mikko.hakala@bore.eu -  Pooling cheaper than using biofuel
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('meeszniehe%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaUE\"","id":"AAMkADllYTVkMjNiLWIyMTUtNDJlMi04ZmEzLTc5MWE1Njg1YzFhZgBGAAAAAAAtEZn6fxGeTa1IW7d-z5k1BwCpQwA2oX5_S6qK2pjvDllMAAAAAAEPAACpQwA2oX5_S6qK2pjvDllMAAAYrea_AAA=","createdDateTime":"2025-04-24T09:30:23Z","lastModifiedDateTime":"2025-04-24T09:30:23Z","changeKey":"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaUE","categories":[],"receivedDateTime":"2025-04-24T09:30:23Z","sentDateTime":"2025-04-24T09:30:23Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB5092F69BAF6E08D00A0D9CD6D6852@AM6PR07MB5092.eurprd07.prod.outlook.com>","subject":" Pooling cheaper than using biofuel","bodyPreview":"Dear Capt. Mikko,\r\n\r\nFuelEU pooling is now trading around EUR 200\u2014 a cheaper, more flexible way 

Creating drafts:  96%|█████████▋| 27/28 [00:33<00:01,  1.25s/it]

Sending email draft to Outlook API: j.burnett@borlines.com -  Pooling cheaper than using biofuel
Response Code: 201
Response Body: {"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users('meeszniehe%40carbonleap.nl')/messages/$entity","@odata.etag":"W/\"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaUG\"","id":"AAMkADllYTVkMjNiLWIyMTUtNDJlMi04ZmEzLTc5MWE1Njg1YzFhZgBGAAAAAAAtEZn6fxGeTa1IW7d-z5k1BwCpQwA2oX5_S6qK2pjvDllMAAAAAAEPAACpQwA2oX5_S6qK2pjvDllMAAAYrea-AAA=","createdDateTime":"2025-04-24T09:30:24Z","lastModifiedDateTime":"2025-04-24T09:30:24Z","changeKey":"CQAAABYAAACpQwA2oX5+S6qK2pjvDllMAAAYmaUG","categories":[],"receivedDateTime":"2025-04-24T09:30:24Z","sentDateTime":"2025-04-24T09:30:24Z","hasAttachments":false,"internetMessageId":"<AM6PR07MB5092B92355D532A92C3B9D8ED6852@AM6PR07MB5092.eurprd07.prod.outlook.com>","subject":" Pooling cheaper than using biofuel","bodyPreview":"Dear James,\r\n\r\nFuelEU pooling is now trading around EUR 200\u2014 a cheaper, more flexible way than

Creating drafts: 100%|██████████| 28/28 [00:34<00:00,  1.22s/it]


Completed email draft creation:
- Successfully created: 28
- Failed: 0
- Total processed: 28
